###2. Write a program that takes input a phrase, a webpage and a number, n; and then outputs all the webpages that match the phrase (exact phrase match), are within ‘n’ distance from the input webpage. The order in which output is displayed should be according to cosine similarity between the initial phrase and the webpage content. The output is should be stored in excel format by the program. [10]

In [1]:
# Importing all necessary libraries
import re
import pandas as pd
import numpy as numpy
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [2]:
#Downloading the NLTK tool kit
nltk.download(['punkt', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
#Taking webpage url as an input from the user
def webpage_url(alert = ''):
  while True:
    url = input(alert)
    if url.startswith('http://') or url.startswith('https://'):
      return url
    else:
      print('Sorry! Given url is invalid. Please try again!\n')
#To get heading of a webpage, from the given url
def webpage_heading(url):
  try:
    html = urlopen(Request(url, headers={'User-Agent':'Mozilla'}))
    bs = BeautifulSoup(html, 'html.parser')
    heading = bs.find('title').get_text()
  except:
    heading = ''
  return heading
# To get the web crawl information levelwise
def level_crawl(pre_url, pre_level):
  m = re.search('https?:\/\/[\w]+\.([A-Za-z_0-9.-]+).*', pre_url)
  parent_domain =  m.group(1) if m else ''
  total_links_found = 0
  links = []
  headings = []
  try:
    html = urlopen(pre_url)
    bs = BeautifulSoup(html, 'html.parser')
    for link in bs.find_all('a'):
      if 'href' in link.attrs and link.attrs['href'].startswith('http'):
        if re.search('https?:\/\/[\w]+\.([A-Za-z_0-9.-]+).*', link.attrs['href']).group(1) != parent_domain:
          links.append(link.attrs['href'])
          heading = webpage_heading(link.attrs['href'])
          headings.append(heading)
  except:
    pass
  all_links_found = len(links)
  parent_url = pre_url
  return pre_level+1, parent_url, all_links_found, links, headings
# Current index of link being crawled
def web_crawl(first_url, level):
  current_index = 0
  list_of_level = []
  list_of_url = []
  list_of_parent_url = []
  list_of_heading = []
  list_of_level.append(0)
  list_of_url.append(first_url)
  list_of_parent_url.append('')
  list_of_heading.append(webpage_heading(first_url))
  if level > 0:
    while current_index < len(list_of_level):
      if list_of_level[current_index] >= level:
        break
      level_present, parent_url, all_links_found, links, headings = level_crawl(pre_url = list_of_url[current_index], pre_level = list_of_level[current_index])
      for i in range(all_links_found):
        list_of_level.append(level_present)
        list_of_url.append(links[i])
        list_of_parent_url.append(parent_url)
        list_of_heading.append(headings[i])

      current_index += 1
  dictn = {'level': list_of_level, 'url': list_of_url, 'parent_url': list_of_parent_url, 'heading': list_of_heading}
  df = pd.DataFrame(dictn)
  df.drop_duplicates(subset=['url'], inplace=True)
  df = df.reset_index(drop=True)
  return df
#Calculate cosine similarity between the initial phrase and  webpage content
def get_cosine_similarity(phrase, url):
  Phrase_1 = phrase.lower()
  Phrase_2 = ''
  try:
    html = urlopen(Request(url, headers={'User-Agent':'Mozilla'}))
    bs = BeautifulSoup(html, 'html.parser')
    for data in bs(['style', 'script']):
      data.decompose()
    text = ' '.join(bs.stripped_strings).lower()
    Phrase_2 = text
  except:
    pass
  # Tokenization
  Phrase_1_token = word_tokenize(Phrase_1)
  Phrase_2_token = word_tokenize(Phrase_2)
  Phrase_2_token  = [re.sub(r'[^a-zA-Z0-9]','',string) for string in Phrase_2_token]
  Phrase_2_token = ' '.join(Phrase_2_token).split()
  stopword = stopwords.words('english')
  list_1 = []
  list_2 = []
  Phrase_1_set = {word for word in Phrase_1_token if not word in stopword} 
  Phrase_2_set = {word for word in Phrase_2_token if not word in stopword}
  vector = Phrase_1_set.union(Phrase_2_set)
  for word in vector:
    if word in Phrase_1_set:
      list_1.append(1)
    else:
      list_2.append(0)
  if word in Phrase_2_set:
      list_2.append(1)
  else:
      list_2.append(0)
  dot_product = 0
  for i in range(len(vector)):
    dot_product += list_1[i]*list_2[i]
  cosine_similarity = dot_product / float((sum(list_1)*sum(list_2))**0.5)
  return cosine_similarity
 #Function to get exact phrase match
def exact_phrase_match(phrase, url):
  matched = False
  try:
    html = urlopen(Request(url, headers={'User-Agent':'Mozilla'}))
    bs = BeautifulSoup(html, 'html.parser')
    # Remove script and style tags
    for data in bs(['style', 'script']):
      data.decompose()
    text = ''.join(bs.stripped_strings) 
    if text.find(phrase) >= 0:
      matched = True
  except:
    pass
    return matched
def calculate_cosine_similarity(first_url, level):
  # Perform web crawling till the given number of 'n' levels is reached
  dataframe = web_crawl(first_url = first_url, level = level)
  dataframe['exact_phrase_match'] = dataframe.apply(lambda row: exact_phrase_match(phrase = phrase, url = row['url']), axis = 1)
  dataframe = dataframe[dataframe['exact_phrase_match'] == True]
  dataframe = dataframe[['level', 'url']]
  dataframe['cosine_similarity'] = 0
  if len(dataframe) == 0:
    return dataframe, 'No webpages matched with the given phrase.'
  # Calculate cosine similarity for the phrase matched webpages
  dataframe['cosine_similarity'] = dataframe.apply(lambda row: get_cosine_similarity(phrase = phrase, url = row['url']),axis = 1)
  dataframe.sort_values(by = ['cosine_similarity'], inplace = True, ascending = False)
  dataframe = dataframe.reset_index(drop=True)
  return dataframe

#Taking Input from the User
url_n = webpage_url('Enter the URL of the Webpage : ')
phrase = input('Enter the Phrase: ')
number = int(input ('Enter the number: '))
dataframe_for_cosine_similarity, alert = calculate_cosine_similarity(first_url = url_n, level = number)
print(alert)
dataframe_for_cosine_similarity

Enter the URL of the Webpage : https://www.javatpoint.com/sql-interview-questions
Enter the Phrase: interview questions
Enter the number: 3
No webpages matched with the given phrase.


Empty DataFrame
Columns: [level, url, cosine_similarity]
Index: []

In [ ]:
_# Converting results of the DataFrame into excel file
dataframe_for_cosine_similarity.to_excel('Results.xlsx', index = False)